<a href="https://colab.research.google.com/github/AbheyTiwari/AbheyTiwari/blob/main/AI_Hallucinations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-learn pandas numpy

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Step 3: Create Training Data (Manually Entered)
# Columns: [D, M, I, A, C] -> Features, H -> Target (0 = No Hallucination, 1 = Hallucination)
data = {
    "D": [0.1, 0.1, 0.5, 0.2, 0.7, 0.3, 0.9, 0.4, 0.2, 0.6],
    "M": [0.8, 0.7, 0.9, 0.6, 0.9, 0.8, 0.7, 0.6, 0.9, 0.9],
    "I": [0.1, 0.5, 0.9, 0.7, 0.9, 0.4, 0.8, 0.6, 0.5, 0.9],
    "A": [0, 0, 0, 0, 0, 0.1, 0.2, 0.3, 0, 0.5],
    "C": [0.9, 0.7, 0.6, 0.8, 0.5, 0.7, 0.4, 0.6, 0.7, 0.3],
    "H": [0, 0, 1, 0, 1, 0, 1, 0, 0, 1]  # 1 = Hallucination, 0 = No Hallucination
}

# Step 4: Convert Data to Pandas DataFrame
df = pd.DataFrame(data)

# Step 5: Split into Features (X) and Target (y)
X = df[["D", "M", "I", "A", "C"]]
y = df["H"]

# Step 6: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 7: Train Logistic Regression Model
model = LogisticRegression()
model.fit(X_train, y_train)

# Step 8: Test Model Accuracy
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

# Step 9: User Input - Predict AI Hallucination
def predict_hallucination():
    print("\nEnter values for the new question:")
    D = float(input("Training Data Quality (0=Perfect, 1=Terrible): "))
    M = float(input("Model Complexity (0=Simple, 1=Complex): "))
    I = float(input("Input Ambiguity (0=Clear, 1=Confusing): "))
    A = float(input("Adversarial Attack Strength (0=None, 1=High): "))
    C = float(input("AI Confidence Score (0=Unsure, 1=Fully Confident): "))

    # Make prediction
    input_features = np.array([[D, M, I, A, C]])
    prediction = model.predict(input_features)[0]

    # Show result
    if prediction == 1:
        print("\n🚨 AI is likely to HALLUCINATE on this question! 🚨")
    else:
        print("\n✅ AI is likely to give a CORRECT answer.")

# Run Prediction
predict_hallucination()

Model Accuracy: 100.00%

Enter values for the new question:
Training Data Quality (0=Perfect, 1=Terrible): 0.2
Model Complexity (0=Simple, 1=Complex): 03.
Input Ambiguity (0=Clear, 1=Confusing): 04.
Adversarial Attack Strength (0=None, 1=High): 0.5
AI Confidence Score (0=Unsure, 1=Fully Confident): 0.1

🚨 AI is likely to HALLUCINATE on this question! 🚨


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [ ]:
import google.generativeai as genai
import re  # Import regex module for extracting numbers

# Set up Gemini API
genai.configure(api_key="your API")

# Initialize weights
weights = {
    "D": 0.2,  # Training data quality
    "M": 0.2,  # Model complexity
    "I": 0.2,  # Input ambiguity
    "A": 0.2,  # Adversarial perturbations
    "C": -0.2  # Confidence score
}

# Function to analyze prompt using Gemini 1.5 Pro
def analyze_prompt(prompt):
    model = genai.GenerativeModel("gemini-1.5-flash")

    response = model.generate_content(f"Analyze this prompt: {prompt}.\n"
                                      "Give numerical values between 0 and 1 for:\n"
                                      "D: Training data quality\n"
                                      "M: Model complexity\n"
                                      "I: Input ambiguity\n"
                                      "A: Adversarial perturbations\n"
                                      "C: AI confidence")

    if not response or not response.text:
        return None  # Handle empty response case

    # Extract numerical values using regex
    scores_dict = {}
    pattern = r"([DMICA]):\s*([\d\.]+)"
    matches = re.findall(pattern, response.text)

    for key, value in matches:
        try:
            scores_dict[key] = float(value)  # Convert to float safely
        except ValueError:
            scores_dict[key] = 0.5  # Default to 0.5 if parsing fails

    return scores_dict if len(scores_dict) == 5 else None  # Ensure all values exist

# Function to calculate hallucination probability
def calculate_hallucination(prompt):
    global weights  # Declare global to modify weights
    scores = analyze_prompt(prompt)

    if scores is None:
        return "Error: No valid response from Gemini"

    H = (weights["D"] * scores["D"] +
         weights["M"] * scores["M"] +
         weights["I"] * scores["I"] +
         weights["A"] * scores["A"] +
         weights["C"] * scores["C"])

    H = max(0, min(H, 1))  # Keep H within 0-1 range
    return H * 100  # Convert to percentage


# Example usage
prompt = input("Please neter the prompt you want to test: ")
hallucination_probability = calculate_hallucination(prompt)

# Check if the function returned an error message
if isinstance(hallucination_probability, str):
    print(hallucination_probability)  # Print the error message directly
else:
    print(f"Hallucination Probability: {hallucination_probability:.2f}%")


Please neter the prompt you want to test: who was ppnavis the 2
Error: No valid response from Gemini
